# imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# `Dot` class definition

In [ ]:
class Dot:
    # todo: maybe the dot class is unnecessary and can be done with a circle when there is one
    def __init__(self, x, y, z, marker='.', size=2, color='white', edge_color='none'):
        self.position = np.array((x, y, z, 1)).T
        self.color = color
        self.size = size
        self.marker = marker
        self.edge_color = edge_color

    def draw(self, ax):
        x = self.position[0]
        y = self.position[1]

        marker_style = dict(marker=self.marker, markersize=self.size, color=self.color, markeredgecolor=self.edge_color)

        ax.plot(x, y, **marker_style)

    def transform(self, transformation):
        self.position = transformation.apply(self.position)
    
    def copy(self):
        c = Dot(*self.position[:3])
        c.color = self.color
        c.size = self.size
        c.marker = self.marker
        c.edge_color = self.edge_color
        return c
    
    def set_color(self, c):
        self.color = c


# `Line` class definition

In [ ]:
class Line:
    def __init__(self, size=1, width=2, color='white'):
        half = size / 2
        self.points = np.array([(-half, 0, 0, 1), (half, 0, 0, 1)]).T
        self.width = width
        self.color = color

    def draw(self, ax):
        xs = self.points[0]
        ys = self.points[1]

        ax.plot(xs, ys, color=self.color, linewidth=self.width)

    def transform(self, transformation):
        self.points = transformation.apply(self.points)

# transformations definitions

In [ ]:


class Transformation:
    def apply(self, points):
        # todo: alterar nome `points`, pois pode ser outra transformacao
        return np.dot(self.matrix, points)


class Translation(Transformation):
    def __init__(self, x=0, y=0, z=0):
        self.matrix = np.array([
            [1, 0, 0, x],
            [0, 1, 0, y],
            [0, 0, 1, z],
            [0, 0, 0, 1]
        ])


class Rotation(Transformation):
    def __init__(self, angle, x=False, y=False):
        # angle: angulo em graus
        # rotacao default em relacao a z
        angle_rad = angle*np.pi / 180
        
        if x:
            self.matrix = self.x_rotation(angle_rad)
        elif y:
            self.matrix = self.y_rotation(angle_rad)
        else:
            self.matrix = self.z_rotation(angle_rad)

    
    def x_rotation(self, deg):
        sin = np.sin(deg)
        cos = np.cos(deg)

        return np.array([
            [1,  0 ,   0 , 0],
            [0, cos, -sin, 0],
            [0, sin,  cos, 0],
            [0,  0,    0,  1]
        ])
    
    def y_rotation(self, deg):
        sin = np.sin(deg)
        cos = np.cos(deg)

        return np.array([
            [ cos, 0, sin, 0],
            [  0,  1,  0 , 0],
            [-sin, 0, cos, 0],
            [  0,  0,  0,  1]
        ])
    
    def z_rotation(self, deg):
        sin = np.sin(deg)
        cos = np.cos(deg)

        return np.array([
            [cos, -sin, 0, 0],
            [sin,  cos, 0, 0],
            [0,     0,  1, 0],
            [0,     0,  0, 1]
        ])


class Scale(Transformation):
    def __init__(self, x=1, y=1, z=1):
        self.matrix = np.array([
            [x, 0, 0, 0],
            [0, y, 0, 0],
            [0, 0, z, 0],
            [0, 0, 0, 1]
        ])


class Perspective(Transformation):
    def __init__(self, x=0, y=0, z=0):
        self.matrix = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [x, y, z, 1]
        ])
    
    def apply(self, points):
        ps = super().apply(points)
        last = ps[3]
        # normaliza
        return ps / last



# experiment 1 - `Dot` with translation and rotation

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)

d1 = Dot(0, 0, 0, color='black', marker='o', size=9)

d2 = d1.copy()
d2.set_color('red')

d2.transform(Translation(x=2))
d2.transform(Rotation(y=90))

perspective = Perspective(x=0, y=0, z=0)
d1.transform(perspective)
d2.transform(perspective)

d1.draw(ax)
d2.draw(ax)

# experiment 2 - perspective

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)

line1 = np.array([(-1, -4, 0, 1), (-1, 4, 0, 1)]).T
line2 = np.array([(1, -4, 0, 1), (1, 4, 0, 1)]).T

translation = Translation(y=2)
line1 = translation.apply(line1)
line2 = translation.apply(line2)

perspective = Perspective(y=0.4)
line1 = perspective.apply(line1)
line2 = perspective.apply(line2)

ax.plot(line1[0], line1[1])
ax.plot(line2[0], line2[1])

# experiment 3 - `Line` with translation, rotation and scaling

In [ ]:
line = Line(color='black')

fig, ax = plt.subplots()

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)

line.transform(Scale(x=2))

print(line.points)

line.draw(ax)